<a href="https://colab.research.google.com/github/teethavattcp/teethavat_port_data/blob/main/06_CustomerMovement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=f9599f808c65210624328ade61d98321a9ef967714b78c34aa5c5f24b90ff31c
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
import pandasql

In [ ]:
import plotly.express as px

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
import datetime as dt
from datetime import datetime

In [ ]:
from pathlib import Path

#Define fn for cleaning raw data

##count_total_duplicates(df)

In [ ]:
def count_total_duplicates(df):
    # Identifying duplicate rows (excluding the first occurrence)
    duplicates = df.duplicated(keep='first')

    # Summing the number of duplicate rows
    total_duplicates = duplicates.sum()

    return total_duplicates

##check_duplication_in_df(df)

In [ ]:
def check_duplication_in_df(df):
  total_duplicates = count_total_duplicates(df)
  if total_duplicates > 0:
    print(f"There are {total_duplicates} duplicate rows in the DataFrame.")
    df.drop_duplicates(inplace=True)
    print("Duplicate rows were dropped.")
    print("New shape of dataframe after dropping duplicate rows:",df.shape)
  elif total_duplicates == 0:
    print("No duplicate rows found.")

## format_number(x)

In [ ]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return '{:.0f}'.format(x)  # Convert to integer if it's a whole number
        else:
            return '{:.2f}'.format(x)  # Keep as float with 2 decimal places
    return x

## explore_df(df_for_eda)

In [ ]:
def explore_df(df_for_eda):
  print("#Check shape")
  print(df_for_eda.shape,"\n")

  print("#Check duplication")
  check_duplication_in_df(df_for_eda)

  print("\n#Check missing value")
  if df_for_eda.isnull().sum().sum() == 0:
    print("No missing values found.")
  else:
    print("Missing values found:", df_for_eda.isnull().sum().sum())
    print("\nNumber of missing values:\n",df_for_eda.isnull().sum())
    print("\nPercentage of missing values:\n",df_for_eda.isnull().sum()/len(df_for_eda)*100)

  print("\n#Check data type")
  print(df_for_eda.info(),"\n")

  print("\n#Check unique value")
  print(df_for_eda.nunique(),"\n")

  print("#List of column name:\n",df_for_eda.columns.tolist())

## explore_classes_in_df(df,df_col_cat)

In [ ]:
def explore_classes_in_df(df,df_col_cat):
  for col in df_col_cat:
    print ("\n-------- %s --------" % col)
    print (df[col].value_counts())

In [ ]:
def explore_distribution_in_df(df,df_col_num):
  for col in df_col_num:
    #histogram plot
    #plt.figure(figsize=(2.5, 1.25))
    print ("\n-------- %s --------" % col)

    # Create the histogram plot to calculate the max count
    hist = sns.histplot(df[col])

    # Calculate the maximum count (frequency)
    max_count = hist.patches[0].get_height()  # Initialize with the first bar's height
    for bar in hist.patches:
        if bar.get_height() > max_count:
            max_count = bar.get_height()

    # Clear the current plot to redraw with y-axis limit
    plt.clf()

    # Create the histogram plot again
    sns.histplot(df[col])

    # Set the maximum limit for the y-axis
    plt.ylim(top=1.3 * max_count)

    # Display the plot
    plt.title(f'Histogram of {col}')
    plt.show()

    #box plot
    #plt.figure(figsize=(2.5, 1.25))
    plt.clf()
    sns.boxplot(x=df[col])
    plt.title(f'Box Plot of {col}')
    plt.show()

## deepexploredf(df,df_col_cat, df_col_num)

In [ ]:
import matplotlib.cm as cm

In [ ]:
def deepexploredf(df,df_col_cat, df_col_num):
  print("##Explore categorical data")
  if len(df_col_cat) == 0:
    print("No categorical column found.")
  else:
    explore_classes_in_df(df,df_col_cat)

  print("\n##Explore numerical data")
  if len(df_col_num) == 0:
    print("No numerical data found.")
  else:
    print("#Check statistical summary")
    print(df[df_col_num].describe().applymap(format_number).transpose().to_markdown())
    print("#Check correlation")
    #plt.figure(figsize = (2,1.25))
    #plt.rcParams.update({'font.size': 10})
    sns.heatmap(df[df_col_num].corr(), annot =True, cmap=cm.coolwarm_r, vmin=-1, vmax=1)
    plt.show()
    #sns.pairplot(df[df_col_num])
    #plt.show()

## export_to_csv(path_of_folder,file_name,df)

In [ ]:
def export_to_csv(path_of_folder,file_name,df):
  filepath = Path(str(path_of_folder)+str(file_name)+'.csv')
  filepath.parent.mkdir(parents=True, exist_ok=True)
  df.to_csv(filepath,mode = 'w', index=False)
  print(str(file_name)+'.csv has been exported successfully.')

In [ ]:
path_folder='/content/drive/MyDrive/DATA_SCI_SKILL/202406_CompleteJourneyProj/data set'+'/'

## explore_ranges_of_num_field(df,df_col_num)

In [ ]:
def explore_ranges_of_num_field(df,list_col_num):
  for col in df[list_col_num]:
    count_less_than_0_butsmallamount = len(df[(df[col] < 0) & (df[col] > -0.01)]) #-0.01 < value < 0
    count_less_than_0_butlargeamount = len(df[df[col] <= -0.01]) #-0.01 =< value
    count_equal_to_0 = len(df[df[col] == 0])
    count_greater_than_0_but_smallamount = len(df[(df[col] > 0)& (df[col] < 0.01)])
    count_greater_than_0_but_largeamount = len(df[(df[col] >= 0.01)])
    print(col,':')
    print(f'Count of values << 0: {count_less_than_0_butlargeamount}')
    print(f'Count of -0.01 < values < 0: {count_less_than_0_butsmallamount}')
    print(f'Count of values = 0: {count_equal_to_0}')
    print(f'Count of 0 < values < 0.01: {count_greater_than_0_but_smallamount}')
    print(f'Count of values >> 0: {count_greater_than_0_but_largeamount}')
    print('\n')

## add_status_column(df, left_col, right_col, status_col='status')

Add a status column to the DataFrame based on the comparison of two columns from left and right DataFrames.
```
 Parameters:
    - df (pd.DataFrame): The DataFrame resulting from an outer join.
    - left_col (str): The column name from the left DataFrame.
    - right_col (str): The column name from the right DataFrame.
    - status_col (str): The name of the new status column. Default is 'status'.

 Returns:
    - pd.DataFrame: The DataFrame with the added status column.

```


In [ ]:
def add_status_column(df, left_col, right_col, status_col='status'):
    conditions = [
        (df[left_col].notna()) & (df[right_col].notna()),  # Found value in both tables
        (df[left_col].notna()) & (df[right_col].isna()),   # Found value only in the left table
        (df[left_col].isna()) & (df[right_col].notna())    # Found value only in the right table
    ]

    choices = ['Found in both', 'Found only in left', 'Found only in right']

    df[status_col] = pd.Series(np.select(conditions, choices, default=None))
    return df

## dict_k2v_v2k(input_dict):

In [ ]:
def dict_k2v_v2k(input_dict):
  output_dict = {}

  for data_type, columns in input_dict.items():
    for column in columns:
        output_dict[column] = data_type

  return output_dict

## replace_valueindf(df,col_key_name,col_val_name,dict_kv_replace)

In [ ]:
def replace_valueindf(df,col_key_name,col_val_name,dict_kv_replace):
  if df[col_key_name] in dict_kv_replace.keys():
    df[col_val_name] = dict_kv_replace[df[col_key_name]]
  return df

# Import dataset

Adhoc analysis considered only last date at 30 Nov 2002 (for fair analysis) --> filter out in powerbi

## df_date

In [ ]:
df_date = pd.read_csv('/content/drive/MyDrive/DATA_SCI_SKILL/202406_CompleteJourneyProj/data set/00_dim_date_withforecast.csv', delimiter=",")
#df_date['TRANS_date'] = pd.to_datetime(df_date['TRANS_date'], format='%d %m %Y')
df_date = df_date[['DAY_ID','MONTH_NO','YEAR_MOD']]
#df_date = df_date[df_date['DAY_ID']<=699]
df_date

,DAY_ID,MONTH_NO,YEAR_MOD
0,1,1,2001
1,2,1,2001
2,3,1,2001
3,4,1,2001
4,5,1,2001
...,...,...,...
815,816,3,2003
816,817,3,2003
817,818,3,2003
818,819,3,2003


## df_fact_trans

In [ ]:
df_fact_trans = pd.read_csv('/content/drive/MyDrive/DATA_SCI_SKILL/202406_CompleteJourneyProj/data set/fact_trans_x_couponredempt.csv')
df_fact_trans = df_fact_trans.rename(columns={'PRODUCT_ID_x': 'PRODUCT_ID'})
df_fact_trans = df_fact_trans[['household_key','DAY','QUANTITY','SALES_VALUE']]
#df_fact_trans = df_fact_trans[df_fact_trans['DAY']<=699]
df_fact_trans

,household_key,DAY,QUANTITY,SALES_VALUE
0,1,51,1,3.99
1,1,51,1,2.99
2,1,51,1,1.09
3,1,51,1,3.71
4,1,51,1,2.79
...,...,...,...,...
2581261,2500,708,1,3.19
2581262,2500,708,1,1.99
2581263,2500,708,1,2.59
2581264,2500,708,1,2.77


# Aggregate data

## df_fact_trans_agg

In [ ]:
df_fact_trans_agg = df_fact_trans.groupby(['household_key','DAY']).agg({'QUANTITY':'sum','SALES_VALUE':'sum'}).reset_index()
df_fact_trans_agg

,household_key,DAY,QUANTITY,SALES_VALUE
0,1,51,34,78.66
1,1,67,14,41.10
2,1,88,13,26.90
3,1,94,32,63.43
4,1,101,20,53.45
...,...,...,...,...
225339,2500,695,34,57.68
225340,2500,698,12,50.54
225341,2500,704,31,73.65
225342,2500,706,6,13.95


In [ ]:
df_fact_trans_agg_month_yr = df_fact_trans_agg.merge(df_date, left_on='DAY', right_on='DAY_ID', how='left')
df_fact_trans_agg_month_yr = df_fact_trans_agg_month_yr.groupby(['household_key','MONTH_NO','YEAR_MOD']).agg({'QUANTITY':'sum','SALES_VALUE':'sum'}).reset_index()
df_fact_trans_agg_month_yr['key_hh_month_yr'] = df_fact_trans_agg_month_yr['household_key'].astype(str) + '_' + df_fact_trans_agg_month_yr['MONTH_NO'].astype(str) + '_' + df_fact_trans_agg_month_yr['YEAR_MOD'].astype(str)
df_fact_trans_agg_month_yr

,household_key,MONTH_NO,YEAR_MOD,QUANTITY,SALES_VALUE,key_hh_month_yr
0,1,1,2002,43,98.70,1_1_2002
1,1,2,2001,34,78.66,1_2_2001
2,1,2,2002,136,260.22,1_2_2002
3,1,3,2001,27,68.00,1_3_2001
4,1,3,2002,111,256.99,1_3_2002
...,...,...,...,...,...,...
45252,2500,10,2002,21121,320.05,2500_10_2002
45253,2500,11,2001,94,283.79,2500_11_2001
45254,2500,11,2002,139,416.28,2500_11_2002
45255,2500,12,2001,128,264.37,2500_12_2001


In [ ]:
df_fact_trans_agg_month_yr = df_fact_trans_agg_month_yr.groupby(['household_key','MONTH_NO','YEAR_MOD','key_hh_month_yr']).agg({'QUANTITY':'sum','SALES_VALUE':'sum'}).reset_index()
df_fact_trans_agg_month_yr

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE
0,1,1,2002,1_1_2002,43,98.70
1,1,2,2001,1_2_2001,34,78.66
2,1,2,2002,1_2_2002,136,260.22
3,1,3,2001,1_3_2001,27,68.00
4,1,3,2002,1_3_2002,111,256.99
...,...,...,...,...,...,...
45252,2500,10,2002,2500_10_2002,21121,320.05
45253,2500,11,2001,2500_11_2001,94,283.79
45254,2500,11,2002,2500_11_2002,139,416.28
45255,2500,12,2001,2500_12_2001,128,264.37


In [ ]:
df_fact_trans_agg_month_yr.describe()

,household_key,MONTH_NO,YEAR_MOD,QUANTITY,SALES_VALUE
count,45257.000000,45257.000000,45257.000000,45257.000000,45257.000000
mean,1248.362286,6.764677,2001.531166,5760.117153,178.037515
std,722.916253,3.291519,0.499033,15303.255364,191.437345
min,1.000000,1.000000,2001.000000,1.000000,0.000000
25%,626.000000,4.000000,2001.000000,22.000000,45.190000
50%,1244.000000,7.000000,2002.000000,63.000000,115.580000
75%,1878.000000,10.000000,2002.000000,225.000000,246.530000
max,2500.000000,12.000000,2002.000000,303045.000000,2598.270000


### Fill 0 for missing month transaction

In [ ]:
#df_master_mm_yy = df_date[['MONTH_NO','YEAR_MOD']].drop_duplicates().reset_index(drop=True)
#df_master_mm_yy

In [ ]:
#df_master_hh = df_fact_trans[['household_key']].drop_duplicates().reset_index(drop=True)
#df_master_hh

In [ ]:
#df_master_hh_mm_yy = df_master_hh.merge(df_master_mm_yy, how='cross')
#df_master_hh_mm_yy['key_hh_month_yr'] = df_master_hh_mm_yy['household_key'].astype(str) + '_' + df_master_hh_mm_yy['MONTH_NO'].astype(str) + '_' + df_master_hh_mm_yy['YEAR_MOD'].astype(str)
#df_master_hh_mm_yy

In [ ]:
#df_fact_trans_agg_month_yr = df_fact_trans_agg_month_yr.merge(df_master_hh_mm_yy, left_on=['key_hh_month_yr'], right_on=['key_hh_month_yr'], how='right')

In [ ]:
#df_fact_trans_agg_month_yr = df_fact_trans_agg_month_yr[['household_key_y', 'MONTH_NO_y','YEAR_MOD_y','QUANTITY', 'SALES_VALUE']]
#df_fact_trans_agg_month_yr = df_fact_trans_agg_month_yr.rename(columns={'household_key_y':'household_key','MONTH_NO_y':'MONTH_NO','YEAR_MOD_y':'YEAR_MOD'})
#df_fact_trans_agg_month_yr['QUANTITY'] = df_fact_trans_agg_month_yr['QUANTITY'].fillna(0)
#df_fact_trans_agg_month_yr['SALES_VALUE'] = df_fact_trans_agg_month_yr['SALES_VALUE'].fillna(0)
#df_fact_trans_agg_month_yr

In [ ]:
df_fact_trans_agg_month_yr['TRANS_date_round'] = pd.to_datetime(df_fact_trans_agg_month_yr[['YEAR_MOD', 'MONTH_NO']].rename(columns={'YEAR_MOD': 'year', 'MONTH_NO': 'month'}).assign(DAY=1))
df_fact_trans_agg_month_yr

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE,TRANS_date_round
0,1,1,2002,1_1_2002,43,98.70,2002-01-01
1,1,2,2001,1_2_2001,34,78.66,2001-02-01
2,1,2,2002,1_2_2002,136,260.22,2002-02-01
3,1,3,2001,1_3_2001,27,68.00,2001-03-01
4,1,3,2002,1_3_2002,111,256.99,2002-03-01
...,...,...,...,...,...,...,...
45252,2500,10,2002,2500_10_2002,21121,320.05,2002-10-01
45253,2500,11,2001,2500_11_2001,94,283.79,2001-11-01
45254,2500,11,2002,2500_11_2002,139,416.28,2002-11-01
45255,2500,12,2001,2500_12_2001,128,264.37,2001-12-01


In [ ]:
eda = df_fact_trans_agg_month_yr.groupby(['household_key']).agg({'MONTH_NO':'count'}).reset_index()
eda = eda.rename(columns={'MONTH_NO':'MONTH_COUNT'})
eda

,household_key,MONTH_COUNT
0,1,23
1,2,17
2,3,19
3,4,15
4,5,17
...,...,...
2495,2496,20
2496,2497,22
2497,2498,20
2498,2499,22


In [ ]:
eda.describe()

,household_key,MONTH_COUNT
count,2500.00000,2500.000000
mean,1250.50000,18.102800
std,721.83216,5.180711
min,1.00000,1.000000
25%,625.75000,15.000000
50%,1250.50000,20.000000
75%,1875.25000,22.000000
max,2500.00000,24.000000


In [ ]:
df_fact_trans_agg_month_yr

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE,TRANS_date_round
0,1,1,2002,1_1_2002,43,98.70,2002-01-01
1,1,2,2001,1_2_2001,34,78.66,2001-02-01
2,1,2,2002,1_2_2002,136,260.22,2002-02-01
3,1,3,2001,1_3_2001,27,68.00,2001-03-01
4,1,3,2002,1_3_2002,111,256.99,2002-03-01
...,...,...,...,...,...,...,...
45252,2500,10,2002,2500_10_2002,21121,320.05,2002-10-01
45253,2500,11,2001,2500_11_2001,94,283.79,2001-11-01
45254,2500,11,2002,2500_11_2002,139,416.28,2002-11-01
45255,2500,12,2001,2500_12_2001,128,264.37,2001-12-01


In [ ]:
df_fact_trans_agg_month_yr.describe()

,household_key,MONTH_NO,YEAR_MOD,QUANTITY,SALES_VALUE,TRANS_date_round
count,45257.000000,45257.000000,45257.000000,45257.000000,45257.000000,45257
mean,1248.362286,6.764677,2001.531166,5760.117153,178.037515,2002-01-04 08:58:29.556532736
min,1.000000,1.000000,2001.000000,1.000000,0.000000,2001-01-01 00:00:00
25%,626.000000,4.000000,2001.000000,22.000000,45.190000,2001-08-01 00:00:00
50%,1244.000000,7.000000,2002.000000,63.000000,115.580000,2002-01-01 00:00:00
75%,1878.000000,10.000000,2002.000000,225.000000,246.530000,2002-07-01 00:00:00
max,2500.000000,12.000000,2002.000000,303045.000000,2598.270000,2002-12-01 00:00:00
std,722.916253,3.291519,0.499033,15303.255364,191.437345,NaN


# Feature Engineering

From EDA, it's safer to use quantity instead of SALES_VALUE as its value can be zero. Furthurmore it's may be appropriate to use 1-month period for churn consideration as all customers has at least one time to purchase 'Food & Beverage' category which would be expected to be purchased frequently.

## Assigning status

### statusall_except_churn

Note: pandasSQL does not have a built-in DATE_DIFF function that accepts a time unit like MONTH directly.

Logic:
- TRANS_date_round_prev IS NULL THEN 'New':

    If there is no previous transaction date (TRANS_date_round_prev is NULL), it assigns the status 'New'. This typically occurs for the first transaction for a household.

- WHEN ... = 1 THEN 'Repeat':

    If the difference between the current transaction date (TRANS_date_round) and the previous transaction date (TRANS_date_round_prev) is exactly 1 month, it assigns the status 'Repeat'.

- WHEN ... > 1 THEN 'Reactivate':

    If the difference between the current and previous transaction dates is more than 1 month, it assigns the status 'Reactivate'. This indicates that the customer is returning after a longer period of inactivity.

In [ ]:
Statusall_except_churn = pandasql.sqldf('''
    SELECT *,
      CASE
      -- Condition of 'New' status: when there is no previous transaction
        WHEN TRANS_date_round_prev IS NULL THEN 'New'

      -- Condition of 'Repeat' status: when the difference between the two dates is 1 month
        WHEN (
            (CAST(strftime('%Y', TRANS_date_round) AS INTEGER) - CAST(strftime('%Y', TRANS_date_round_prev) AS INTEGER)) * 12 +
            (CAST(strftime('%m', TRANS_date_round) AS INTEGER) - CAST(strftime('%m', TRANS_date_round_prev) AS INTEGER))
        ) = 1 THEN 'Repeat'

      -- Condition of 'Reactivate' status: when the difference is greater than 1 month
        WHEN (
            (CAST(strftime('%Y', TRANS_date_round) AS INTEGER) - CAST(strftime('%Y', TRANS_date_round_prev) AS INTEGER)) * 12 +
            (CAST(strftime('%m', TRANS_date_round) AS INTEGER) - CAST(strftime('%m', TRANS_date_round_prev) AS INTEGER))
        ) > 1 THEN 'Reactivate'
      ELSE NULL
      END AS Status
    FROM (
        SELECT *,
              LAG(TRANS_date_round, 1) OVER (PARTITION BY household_key ORDER BY TRANS_date_round) AS TRANS_date_round_prev
        FROM df_fact_trans_agg_month_yr
    )
''')
Statusall_except_churn

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE,TRANS_date_round,TRANS_date_round_prev,Status
0,1,2,2001,1_2_2001,34,78.66,2001-02-01 00:00:00.000000,None,New
1,1,3,2001,1_3_2001,27,68.00,2001-03-01 00:00:00.000000,2001-02-01 00:00:00.000000,Repeat
2,1,4,2001,1_4_2001,75,167.19,2001-04-01 00:00:00.000000,2001-03-01 00:00:00.000000,Repeat
3,1,5,2001,1_5_2001,118,271.10,2001-05-01 00:00:00.000000,2001-04-01 00:00:00.000000,Repeat
4,1,6,2001,1_6_2001,111,237.93,2001-06-01 00:00:00.000000,2001-05-01 00:00:00.000000,Repeat
...,...,...,...,...,...,...,...,...,...
45252,2500,8,2002,2500_8_2002,20887,459.01,2002-08-01 00:00:00.000000,2002-07-01 00:00:00.000000,Repeat
45253,2500,9,2002,2500_9_2002,21268,350.49,2002-09-01 00:00:00.000000,2002-08-01 00:00:00.000000,Repeat
45254,2500,10,2002,2500_10_2002,21121,320.05,2002-10-01 00:00:00.000000,2002-09-01 00:00:00.000000,Repeat
45255,2500,11,2002,2500_11_2002,139,416.28,2002-11-01 00:00:00.000000,2002-10-01 00:00:00.000000,Repeat


In [ ]:
print(Statusall_except_churn.head(100).to_markdown())

|    |   household_key |   MONTH_NO |   YEAR_MOD |   key_hh_month_yr |   QUANTITY |   SALES_VALUE | TRANS_date_round           | TRANS_date_round_prev      | Status     |
|---:|----------------:|-----------:|-----------:|------------------:|-----------:|--------------:|:---------------------------|:---------------------------|:-----------|
|  0 |               1 |          2 |       2001 |          1_2_2001 |         34 |         78.66 | 2001-02-01 00:00:00.000000 |                            | New        |
|  1 |               1 |          3 |       2001 |          1_3_2001 |         27 |         68    | 2001-03-01 00:00:00.000000 | 2001-02-01 00:00:00.000000 | Repeat     |
|  2 |               1 |          4 |       2001 |          1_4_2001 |         75 |        167.19 | 2001-04-01 00:00:00.000000 | 2001-03-01 00:00:00.000000 | Repeat     |
|  3 |               1 |          5 |       2001 |          1_5_2001 |        118 |        271.1  | 2001-05-01 00:00:00.000000 | 2001-04-01 00:00

In [ ]:
Statusall_except_churn.head(5)

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE,TRANS_date_round,TRANS_date_round_prev,Status
0,1,2,2001,1_2_2001,34,78.66,2001-02-01 00:00:00.000000,None,New
1,1,3,2001,1_3_2001,27,68.00,2001-03-01 00:00:00.000000,2001-02-01 00:00:00.000000,Repeat
2,1,4,2001,1_4_2001,75,167.19,2001-04-01 00:00:00.000000,2001-03-01 00:00:00.000000,Repeat
3,1,5,2001,1_5_2001,118,271.10,2001-05-01 00:00:00.000000,2001-04-01 00:00:00.000000,Repeat
4,1,6,2001,1_6_2001,111,237.93,2001-06-01 00:00:00.000000,2001-05-01 00:00:00.000000,Repeat


### Status_only_churn

Churn status corresponds with the following condition:
- n_churn_M > 1: Filters for churn periods that are more than 1 month.
- n_churn_M IS NULL: Includes cases where the churn period is in the latest month (no subsequent transaction).
- TRANS_date_round < (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr): Excludes records with the latest transaction date to avoid including the current period.

In [ ]:
Status_only_churn = pandasql.sqldf('''
    SELECT household_key,
           DATE(TRANS_date_round, '+1 month') AS TRANS_date_round_churnreported,"Churn" AS Status,
           -- Change n_churn_M to be Integer for transforming to layer of churn
           CASE
               WHEN n_churn_M > 0 THEN n_churn_M - 1
               WHEN DATE(TRANS_date_round, '+1 month') = (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr) THEN 1
               WHEN DATE(TRANS_date_round, '+1 month') < (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr)
               THEN (CAST(strftime('%Y', (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr)) AS INT) * 12 +
                 CAST(strftime('%m', (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr)) AS INT)) -
                (CAST(strftime('%Y', DATE(TRANS_date_round, '+1 month')) AS INT) * 12 +
                 CAST(strftime('%m', DATE(TRANS_date_round, '+1 month')) AS INT)) + 1
           ELSE 0
       END AS n_churn_M_int,

           CASE
               WHEN n_churn_M > 0 THEN n_churn_M - 1
               WHEN DATE(TRANS_date_round, '+1 month') <= (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr) THEN 0
               ELSE NULL
           END AS n_churn_btw_M

    FROM (
        SELECT *,
               (CAST(strftime('%Y', LEAD(TRANS_date_round, 1) OVER (PARTITION BY household_key ORDER BY TRANS_date_round)) AS INT) * 12 +
            CAST(strftime('%m', LEAD(TRANS_date_round, 1) OVER (PARTITION BY household_key ORDER BY TRANS_date_round)) AS INT)) -
           (CAST(strftime('%Y', TRANS_date_round) AS INT) * 12 +
            CAST(strftime('%m', TRANS_date_round) AS INT)) AS n_churn_M
        FROM df_fact_trans_agg_month_yr
    )
    WHERE ((n_churn_M > 1) --Churn btw HY
        OR (n_churn_M IS NULL)) --Churn in lastest month*
        AND TRANS_date_round < (SELECT MAX(TRANS_date_round) FROM df_fact_trans_agg_month_yr) --Excluding last current period*

''')
Status_only_churn

,household_key,TRANS_date_round_churnreported,Status,n_churn_M_int,n_churn_btw_M
0,2,2002-01-01,Churn,1,1
1,2,2002-07-01,Churn,1,1
2,2,2002-11-01,Churn,2,0
3,3,2002-08-01,Churn,2,2
4,4,2002-03-01,Churn,1,1
...,...,...,...,...,...
4832,2495,2002-04-01,Churn,1,1
4833,2495,2002-06-01,Churn,4,4
4834,2495,2002-11-01,Churn,2,0
4835,2496,2002-12-01,Churn,1,0


In [ ]:
Status_only_churn[Status_only_churn['household_key']==15]

,household_key,TRANS_date_round_churnreported,Status,n_churn_M_int,n_churn_btw_M
30,15,2001-04-01,Churn,1,1
31,15,2001-06-01,Churn,3,3
32,15,2001-12-01,Churn,2,2
33,15,2002-03-01,Churn,1,1


In [ ]:
Statusall_except_churn[Statusall_except_churn['household_key']==15]

,household_key,MONTH_NO,YEAR_MOD,key_hh_month_yr,QUANTITY,SALES_VALUE,TRANS_date_round,TRANS_date_round_prev,Status
229,15,1,2001,15_1_2001,2,4.98,2001-01-01,None,New
230,15,2,2001,15_2_2001,62,92.26,2001-02-01,2001-01-01 00:00:00.000000,Repeat
231,15,3,2001,15_3_2001,11,16.16,2001-03-01,2001-02-01 00:00:00.000000,Repeat
232,15,5,2001,15_5_2001,2,7.58,2001-05-01,2001-03-01 00:00:00.000000,Reactivate
233,15,9,2001,15_9_2001,47,40.88,2001-09-01,2001-05-01 00:00:00.000000,Reactivate
234,15,10,2001,15_10_2001,40,103.41,2001-10-01,2001-09-01 00:00:00.000000,Repeat
235,15,11,2001,15_11_2001,8,16.15,2001-11-01,2001-10-01 00:00:00.000000,Repeat
236,15,2,2002,15_2_2002,18,40.85,2002-02-01,2001-11-01 00:00:00.000000,Reactivate
237,15,4,2002,15_4_2002,88,155.03,2002-04-01,2002-02-01 00:00:00.000000,Reactivate
238,15,5,2002,15_5_2002,39,69.99,2002-05-01,2002-04-01 00:00:00.000000,Repeat


The result seems to be correct

## union all status

###Status_all

In [ ]:
Statusall_except_churn['TRANS_date_round'] = pd.to_datetime(Statusall_except_churn['TRANS_date_round'])
Status_only_churn['TRANS_date_round_churnreported'] = pd.to_datetime(Status_only_churn['TRANS_date_round_churnreported'])

In [ ]:
Statusall = pandasql.sqldf('''
  --Union all status
  SELECT household_key, TRANS_date_round , QUANTITY, SALES_VALUE, Status, 0 AS n_churn_M_int, 0 AS n_churn_btw_M
  FROM Statusall_except_churn

  UNION

  SELECT household_key, TRANS_date_round_churnreported AS TRANS_date_round, CAST(0 AS INT) AS QUANTITY, CAST(0 AS DECIMAL(10, 2)) AS SALES_VALUE, Status, n_churn_M_int, n_churn_btw_M
  FROM Status_only_churn
''')
Statusall['TRANS_date_round'] = pd.to_datetime(Statusall['TRANS_date_round'])
Statusall

,household_key,TRANS_date_round,QUANTITY,SALES_VALUE,Status,n_churn_M_int,n_churn_btw_M
0,1,2001-02-01,34,78.66,New,0,0
1,1,2001-03-01,27,68.00,Repeat,0,0
2,1,2001-04-01,75,167.19,Repeat,0,0
3,1,2001-05-01,118,271.10,Repeat,0,0
4,1,2001-06-01,111,237.93,Repeat,0,0
...,...,...,...,...,...,...,...
50089,2500,2002-08-01,20887,459.01,Repeat,0,0
50090,2500,2002-09-01,21268,350.49,Repeat,0,0
50091,2500,2002-10-01,21121,320.05,Repeat,0,0
50092,2500,2002-11-01,139,416.28,Repeat,0,0


In [ ]:
list_hh_key = list(Statusall['household_key'].unique())
list_hh_key[0:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
list_hh_key = list(Statusall['household_key'].unique())
for hh in list_hh_key[0:20]:
  print(Statusall[Statusall['household_key']==hh].to_markdown())

|    |   household_key | TRANS_date_round    |   QUANTITY |   SALES_VALUE | Status   |   n_churn_M_int |   n_churn_btw_M |
|---:|----------------:|:--------------------|-----------:|--------------:|:---------|----------------:|----------------:|
|  0 |               1 | 2001-02-01 00:00:00 |         34 |         78.66 | New      |               0 |               0 |
|  1 |               1 | 2001-03-01 00:00:00 |         27 |         68    | Repeat   |               0 |               0 |
|  2 |               1 | 2001-04-01 00:00:00 |         75 |        167.19 | Repeat   |               0 |               0 |
|  3 |               1 | 2001-05-01 00:00:00 |        118 |        271.1  | Repeat   |               0 |               0 |
|  4 |               1 | 2001-06-01 00:00:00 |        111 |        237.93 | Repeat   |               0 |               0 |
|  5 |               1 | 2001-07-01 00:00:00 |         71 |        147.05 | Repeat   |               0 |               0 |
|  6 |          

## Assign additional columns: dim_is_status

### Is_churn	Is_reactive_from_churn_btw_month

In [ ]:
dim_is_status = pandasql.sqldf('''
    WITH Statusall_agg AS(
    SELECT household_key, SUM(n_churn_M_int) AS Total_n_churn_M_int, SUM(n_churn_btw_M) AS Total_n_churn_btw_M, MAX(n_churn_btw_M) AS Max_n_churn_btw_M
    FROM Statusall
    GROUP BY household_key)

    SELECT household_key, Max_n_churn_btw_M,
          CASE WHEN Total_n_churn_M_int > 0 THEN 1 ELSE 0 END AS Is_churn,
          CASE WHEN Total_n_churn_btw_M > 0 THEN 1 ELSE 0 END AS Is_reactive_from_churn_btw_month
    FROM Statusall_agg
''')
dim_is_status

,household_key,Max_n_churn_btw_M,Is_churn,Is_reactive_from_churn_btw_month
0,1,0,0,0
1,2,1,1,1
2,3,2,1,1
3,4,2,1,1
4,5,2,1,1
...,...,...,...,...
2495,2496,0,1,0
2496,2497,0,0,0
2497,2498,1,1,1
2498,2499,0,0,0


### Is_Churn_latest

In [ ]:
Churn_latest_hh = pandasql.sqldf('''
    SELECT DISTINCT household_key
    FROM Statusall
    WHERE Status = 'Churn' AND TRANS_date_round = (SELECT MAX(TRANS_date_round) FROM Statusall)
''')
Churn_latest_hh

,household_key
0,9
1,10
2,16
3,17
4,21
...,...
535,2480
536,2482
537,2487
538,2490


In [ ]:
dim_is_status = pandasql.sqldf('''
    SELECT X.*,
          CASE WHEN Churn_latest_hh.household_key IS NOT NULL THEN 1 ELSE 0 END AS Is_Churn_latest
    FROM dim_is_status AS X
    LEFT JOIN Churn_latest_hh
    ON X.household_key = Churn_latest_hh.household_key
''')
dim_is_status

,household_key,Max_n_churn_btw_M,Is_churn,Is_reactive_from_churn_btw_month,Is_Churn_latest
0,1,0,0,0,0
1,2,1,1,1,0
2,3,2,1,1,0
3,4,2,1,1,0
4,5,2,1,1,0
...,...,...,...,...,...
2495,2496,0,1,0,1
2496,2497,0,0,0,0
2497,2498,1,1,1,0
2498,2499,0,0,0,0


In [ ]:
Active_latest_hh = pandasql.sqldf('''
    SELECT DISTINCT household_key
    FROM Statusall
    WHERE (Status = 'New' AND TRANS_date_round = (SELECT MAX(TRANS_date_round) FROM Statusall))
        OR (Status = 'Repeat' AND TRANS_date_round = (SELECT MAX(TRANS_date_round) FROM Statusall))
        OR (Status = 'Reactivate' AND TRANS_date_round = (SELECT MAX(TRANS_date_round) FROM Statusall))
''')
Active_latest_hh

,household_key
0,1
1,3
2,5
3,6
4,7
...,...
1593,2494
1594,2497
1595,2498
1596,2499


In [ ]:
dim_is_status = pandasql.sqldf('''
    SELECT X.*,
          CASE WHEN Y.household_key IS NOT NULL THEN 1 ELSE 0 END AS Is_Active_latest
    FROM dim_is_status AS X
    LEFT JOIN Active_latest_hh AS Y
    ON X.household_key = Y.household_key
''')
dim_is_status

,household_key,Max_n_churn_btw_M,Is_churn,Is_reactive_from_churn_btw_month,Is_Churn_latest,Is_Active_latest
0,1,0,0,0,0,1
1,2,1,1,1,0,0
2,3,2,1,1,0,1
3,4,2,1,1,0,0
4,5,2,1,1,0,1
...,...,...,...,...,...,...
2495,2496,0,1,0,1,0
2496,2497,0,0,0,0,1
2497,2498,1,1,1,0,1
2498,2499,0,0,0,0,1


In [ ]:
Active_latestprev1month_hh = pandasql.sqldf('''
    SELECT DISTINCT household_key
    FROM Statusall
    WHERE (Status = 'New' AND DATE(TRANS_date_round) = (SELECT DATE(MAX(TRANS_date_round), '-1 month')  FROM Statusall))
        OR (Status = 'Repeat' AND DATE(TRANS_date_round) = (SELECT DATE(MAX(TRANS_date_round), '-1 month')  FROM Statusall))
        OR (Status = 'Reactivate' AND DATE(TRANS_date_round) = (SELECT DATE(MAX(TRANS_date_round), '-1 month')  FROM Statusall))
''')
Active_latestprev1month_hh

,household_key
0,1
1,3
2,5
3,6
4,7
...,...
2039,2496
2040,2497
2041,2498
2042,2499


In [ ]:
dim_is_status = pandasql.sqldf('''
    SELECT X.*,
          CASE WHEN Y.household_key IS NOT NULL THEN 1 ELSE 0 END AS Is_Active_latestprev1month
    FROM dim_is_status AS X
    LEFT JOIN Active_latestprev1month_hh AS Y
    ON X.household_key = Y.household_key
''')
dim_is_status

,household_key,Max_n_churn_btw_M,Is_churn,Is_reactive_from_churn_btw_month,Is_Churn_latest,Is_Active_latest,Is_Active_latestprev1month
0,1,0,0,0,0,1,1
1,2,1,1,1,0,0,0
2,3,2,1,1,0,1,1
3,4,2,1,1,0,0,0
4,5,2,1,1,0,1,1
...,...,...,...,...,...,...,...
2495,2496,0,1,0,1,0,1
2496,2497,0,0,0,0,1,1
2497,2498,1,1,1,0,1,1
2498,2499,0,0,0,0,1,1


In [ ]:
Churn_latestprev1month_hh = pandasql.sqldf('''
    SELECT DISTINCT household_key
    FROM Statusall
    WHERE (Status = 'Churn' AND DATE(TRANS_date_round) = (SELECT DATE(MAX(TRANS_date_round), '-1 month')  FROM Statusall))
''')
Churn_latestprev1month_hh

,household_key
0,2
1,62
2,73
3,74
4,76
...,...
200,2420
201,2430
202,2454
203,2468


In [ ]:
dim_is_status = pandasql.sqldf('''
    SELECT X.*,
          CASE WHEN Y.household_key IS NOT NULL THEN 1 ELSE 0 END AS Is_Churn_latestprev1month
    FROM dim_is_status AS X
    LEFT JOIN Churn_latestprev1month_hh AS Y
    ON X.household_key = Y.household_key
''')
dim_is_status

,household_key,Max_n_churn_btw_M,Is_churn,Is_reactive_from_churn_btw_month,Is_Churn_latest,Is_Active_latest,Is_Active_latestprev1month,Is_Churn_latestprev1month
0,1,0,0,0,0,1,1,0
1,2,1,1,1,0,0,0,1
2,3,2,1,1,0,1,1,0
3,4,2,1,1,0,0,0,0
4,5,2,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...
2495,2496,0,1,0,1,0,1,0
2496,2497,0,0,0,0,1,1,0
2497,2498,1,1,1,0,1,1,0
2498,2499,0,0,0,0,1,1,0


เอาพวกนี้มาสร้างเป็น filter 1 อัน โดยตั้ง condition

# Export result

In [ ]:
export_to_csv(path_folder,'06_fact_cust_status',Statusall)

06_fact_cust_status.csv has been exported successfully.


In [ ]:
export_to_csv(path_folder,'06_dim_is_status_filter',dim_is_status)

06_dim_is_status_filter.csv has been exported successfully.
